# Scraping list from an searched flipkart product

### Project Outline:
- For example purpose we are going to scrape this URL:https://www.flipkart.com/search?q=mobiles+under+20000&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_3_8_na_na_na&otracker1=AS_Query_OrganicAutoSuggest_3_8_na_na_na&as-pos=3&as-type=RECENT&suggestionId=mobiles+under+20000&requestId=db87adf1-4f6c-4884-b26e-6e9053b9f6f8&as-backfill=on
- If we look carefully in the URL we can see that I searched for "Mobile Phones under 20000" in the Amazon search bar
- We will extract all the data which is displayed to us in the first page of this flipkart search
- The data which we will extract is the name of the device, price of the device, no of reviews(stars? maybe?!) and URL to that device so one can go and know deeper about that devce.


In [1]:
pip install bs4 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests

In [4]:
products_url="https://www.flipkart.com/search?q=mobiles+under+20000&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_3_8_na_na_na&otracker1=AS_Query_OrganicAutoSuggest_3_8_na_na_na&as-pos=3&as-type=RECENT&suggestionId=mobiles+under+20000&requestId=db87adf1-4f6c-4884-b26e-6e9053b9f6f8&as-backfill=on"

In [5]:
response=requests.get(products_url)

In [6]:
response.status_code

200

In [7]:
page_contents = response.text

In [8]:
doc = BeautifulSoup(page_contents,'html.parser')

In [9]:
heading_class = "_4rR01T"
heading_tags = doc.findAll('div',class_= heading_class)
name_text = []

In [10]:
heading_tags[:4]

[<div class="_4rR01T">realme 8i (Space Black, 128 GB)</div>,
 <div class="_4rR01T">realme 9i (Prism Black, 128 GB)</div>,
 <div class="_4rR01T">realme 9i (Prism Blue, 128 GB)</div>,
 <div class="_4rR01T">MOTOROLA G40 Fusion (Dynamic Gray, 128 GB)</div>]

In [11]:
price_class = "_30jeq3 _1_WHN1"
price_tags = doc.findAll('div', class_=price_class)
price_text = [] 

In [12]:
price_tags[:4]

[<div class="_30jeq3 _1_WHN1">₹15,999</div>,
 <div class="_30jeq3 _1_WHN1">₹15,999</div>,
 <div class="_30jeq3 _1_WHN1">₹15,999</div>,
 <div class="_30jeq3 _1_WHN1">₹16,499</div>]

In [13]:
stars_class= "_3LWZlK"
stars_tags = doc.findAll('div', class_=stars_class)
stars_text = [] 
stars_tags[:4]

[<div class="_3LWZlK">4.4</div>,
 <div class="_3LWZlK">4.4</div>,
 <div class="_3LWZlK">4.4</div>,
 <div class="_3LWZlK">4.3</div>]

In [14]:
reviews_class = "_2_R_DZ"

In [15]:
reviews_tags = doc.findAll('span', class_=reviews_class)
reviews_text = []
reviews_tags[:4]

[<span class="_2_R_DZ"><span><span>55,338 Ratings </span><span class="_13vcmD">&amp;</span><span> 3,762 Reviews</span></span></span>,
 <span class="_2_R_DZ"><span><span>1,135 Ratings </span><span class="_13vcmD">&amp;</span><span> 101 Reviews</span></span></span>,
 <span class="_2_R_DZ"><span><span>1,135 Ratings </span><span class="_13vcmD">&amp;</span><span> 101 Reviews</span></span></span>,
 <span class="_2_R_DZ"><span><span>37,122 Ratings </span><span class="_13vcmD">&amp;</span><span> 4,024 Reviews</span></span></span>]

In [16]:
url_class = "_1fQZEK"
base_url = "https://www.flipkart.com"
url_tags = doc.findAll('a', class_=url_class)
url_product = []
for i in range(len(url_tags)):
    url_product.append(base_url+url_tags[i]['href'])


In [17]:
for i in range(len(reviews_tags)):
        price_text.append(price_tags[i].text)
        name_text.append(heading_tags[i].text)
        stars_text.append(stars_tags[i].text)
        reviews_text.append(reviews_tags[i].text)
        reviews_text[i]=reviews_text[i].replace("\xa0"," ")


- What we did in the above for loop is we made 4 different lists to store all the data in string format and is the list have the data of that particular product and every other list have its rest of the data respective to its indices

In [20]:
product_dict={'name':name_text,'price':price_text,'stars':stars_text,'reviews':reviews_text,'url':url_product}
import pandas as pd
products_df=pd.DataFrame(product_dict)
products_df[:4]

,name,price,stars,reviews,url
0,"realme 8i (Space Black, 128 GB)","₹15,999",4.4,"55,338 Ratings & 3,762 Reviews",https://www.flipkart.com/realme-8i-space-black...
1,"realme 9i (Prism Black, 128 GB)","₹15,999",4.4,"1,135 Ratings & 101 Reviews",https://www.flipkart.com/realme-9i-prism-black...
2,"realme 9i (Prism Blue, 128 GB)","₹15,999",4.4,"1,135 Ratings & 101 Reviews",https://www.flipkart.com/realme-9i-prism-blue-...
3,"MOTOROLA G40 Fusion (Dynamic Gray, 128 GB)","₹16,499",4.3,"37,122 Ratings & 4,024 Reviews",https://www.flipkart.com/motorola-g40-fusion-d...


In [21]:
products_df.to_csv('products.csv')